In [31]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
from torchvision.io import read_image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

import pandas as pd
import matplotlib.pyplot as plt
import json
import string

from PIL import Image

from typing import Any

In [32]:
root_dir = './dataset/chest_xray/'
train_data_path = root_dir + 'train/'
test_data_path = root_dir + 'test/'
val_data_path = root_dir + 'val/'

In [33]:
transforms =  transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [34]:
class PneumoniaDataset(Dataset):
    def __init__(self, root_dir: str, *args, **kwargs) -> None:
        self.train_data = None
        self.test_data = None
        self.val_data = None
        pass

    def __len__(self,) -> int:
        pass

    def __getitem__(self,) -> Any:
        pass

In [35]:
train_dataset = torchvision.datasets.ImageFolder(
    root=train_data_path,
    transform=transforms
)
test_dataset = torchvision.datasets.ImageFolder(
    root=test_data_path,
    transform=transforms
)
val_dataset = torchvision.datasets.ImageFolder(
    root=val_data_path,
    transform=transforms
)
BATCH_SIZE = 16

In [36]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
print(len(train_loader.dataset))

5216


In [37]:
# train_features, train_labels = next(iter(train_loader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze().permute(1,2,0)
# label = train_labels[0]
# plt.imshow(img)
# plt.show()
# print(f"Label: {label}")

In [38]:

class Network(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(44944, 240)
        self.fc2 = nn.Linear(240, 84)
        self.fc3 = nn.Linear(84, 2)
        self.drop = nn.Dropout(0.3)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.drop(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
net = Network()
net.to(device)

cuda


Network(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=240, bias=True)
  (fc2): Linear(in_features=240, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
)

In [39]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.01)

In [40]:
for epoch in range(10):  # loop over the dataset multiple times
    net.train()
    training_loss = 0.0
    valid_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        optimizer.zero_grad()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        training_loss += loss.data.item() * inputs.size(0)
    training_loss /= len(train_loader.dataset)

    net.eval()
    num_correct = 0
    num_examples = 0
    for i, data in enumerate(val_loader, 0):
        optimizer.zero_grad()
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        valid_loss += loss.data.item() * inputs.size(0)
        correct = torch.eq(torch.max(F.softmax(outputs, dim=1), dim=1)[1], 
                               labels)
        num_correct += torch.sum(correct).item()
        num_examples += correct.shape[0]
    valid_loss /= len(val_loader.dataset)
    print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))
print('Finished Training')

KeyboardInterrupt: 